In [9]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import re

In [31]:
#크롤링을 통해 책의 고유 ID를 수집
num = 1
itemids = []
while True:
    url = 'https://www.aladin.co.kr/search/wsearchresult.aspx?SearchTarget=Book&KeyWord=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&KeyRecentPublish=0&OutStock=0&ViewType=Detail&SortOrder=11&CustReviewCount=0&CustReviewRank=0&KeyFullWord=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&KeyLastWord=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&PriceFilterMax=-1&CategorySearch=&chkKeyTitle=&chkKeyAuthor=&chkKeyPublisher=&chkKeyISBN=&chkKeyTag=&chkKeyTOC=&chkKeySubject=&ViewRowCount=50&page='
    page = requests.get(url +str(num))
    soup = BeautifulSoup(page.text, 'html.parser')
    
    bookInfo = soup.find_all('div', attrs={'class':'ss_book_box'}) # div테그 중 class명이 ss_book_box인 것들을 모두 가져온다
    
    if len(bookInfo) <= 0:
        break
    
    for element in bookInfo:
        if 'itemid' in element.attrs:
            itemids.append(element.attrs['itemid'])
    
    num += 1
print('Done!')

Done!


In [ ]:
# DataFrame 생성
columns = ['제목','부제', '저자', '출판일', '출판사', '소개', '목차', '출판사제공','가격'] #데이터 프레임의 컬럼명 
dataFrame = pd.DataFrame(columns=colmns) #dataFrame을 만들고 지정한 column명으로 설정

In [ ]:
# 책의 상세 페이지에 접근하여 데이터 크롤링


for i in itemids:
    url = 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId='
    page = requests.get(url+str(i))
    soup = BeautifulSoup(page.text, 'html.parser')
    
    

In [36]:
url = 'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId='
page = requests.get(url+str(237677114))
soup = BeautifulSoup(page.text, 'html.parser')

#데이터 수집
title = soup.find('a', attrs={'class':'Ere_bo_title'}).text #제목
subTitle = soup.find('a', attrs={'class': 'Ere_sub1_title'}).text #부제

author = soup.find('a', attrs={'class':'Ere_sub2_tile'}).text #작가(1작가만 가져옴)

#출판사 
sub = soup.find_all('a', attrs={'class':'Ere_sub2_title'})
for info in sub:
    if re.compile('Publisher').search(info.attrs['href']):
        publisher = info.text 
        
#출판일        
date = soup.find('li', attrs={'class':'Ere_sub2_title'}).text
compliedDate = re.compile('\d{4}\-\d\d\-\d\d')
date = compliedDate.search(date).group()

#가격
price = soup.find('div', attrs={'class':'Ritem'}).text
compiledPrice = re.compile('\d*,?\d*')
price = compiledPrice.search(price).group()

#목차
introduction = soup.find('div', attrs={'class':'Ere_prod_mconts_R'}).text

In [94]:
introduction = soup.('', attrs={'class':'Ere_prod_mconts_box'})

In [97]:
contents = soup.find('a', attrs={'href':"javascript:fn_hide_introduce_TOC('TOC')"})

In [98]:
contents

In [95]:
for i in introduction:
    print(i.text)


기본정보 
기본정보 


952쪽183*235mm1809gISBN : 9791162242964



주제 분류
신간알림 신청



국내도서 > 컴퓨터/모바일 > 컴퓨터 공학 > 데이터베이스 개론 접기국내도서 > 컴퓨터/모바일 > 컴퓨터 공학 > 자료구조/알고리즘 접기국내도서 > 컴퓨터/모바일 > 컴퓨터 공학 > 전산수학(SPSS/MATLAB) 접기국내도서 > 컴퓨터/모바일 > 프로그래밍 언어 > 파이썬 접기






출판사 제공 북트레일러출판사 제공 북트레일러

이벤트 
이벤트 




   한빛미디어 프로그래밍 단행본 구매 시, <핸즈온 머신러닝> 마우스패드     수험서.외국어.컴퓨터.대학교재 - 라떼 유리컵/스톱워치/본투리드 볼펜/분철 라벨스틱     IT 신간 추천도서 3만원 이상 구매 시, 우드캡 보틀  


In [47]:
title = soup.find_all('a', attrs={'href':re.compile('https://www.aladin.co.kr/search/wsearchresult.aspx?PublisherSearch=')})

In [49]:
title = soup.find_all('a', attrs={'class':'Ere_sub2_title'})

In [56]:
title[0].attrs['href']

'/search/wsearchresult.aspx?AuthorSearch=%ec%98%a4%eb%a0%90%eb%a6%ac%ec%95%99+%ec%a0%9c%eb%a1%b1@6250046&BranchType=1'

In [58]:
sub = soup.find_all('a', attrs={'class':'Ere_sub2_title'})
for info in sub:
    if re.compile('Publisher').search(info.attrs['href']):
        publisher = info.text

In [86]:
price = soup.find('div', attrs={'class':'Ritem'}).text
compiledPrice = re.compile('\d*,?\d*')

In [88]:
price = compiledPrice.search(price).group()

'55,000'

In [62]:
num='rasdfwefasdfw2020-05-04asdfwfsasdf'


In [70]:
a=re.compile('\d{4}\-\d\d\-\d\d')

In [74]:
a.search(date).group()

'2020-05-04'

58054191
81518846